<h1>Fashion MNIST Classification</h1>

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from sklearn.metrics import recall_score,precision_score,f1_score
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.set_random_seed(0)
from tensorflow import keras

2022-08-23 16:19:54.861660: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
data_train = pd.read_csv("/home/zaripovdm/Рабочий стол/archive/fashion-mnist_train.csv")

In [3]:
train_features=[]
train_labels = []
for i in range(60000):
    train_features.append(np.array(data_train.iloc[i][1:]).reshape(28,28))
    train_labels.append(data_train.iloc[i][0])

In [4]:
train_features = np.array(train_features)
train_labels = np.array(train_labels).reshape(-1,1)
print("Train Features : ",train_features.shape)
print("Train Labels : ",train_labels.shape)

Train Features :  (60000, 28, 28)
Train Labels :  (60000, 1)


In [5]:
data_test = pd.read_csv("/home/zaripovdm/Рабочий стол/archive/fashion-mnist_test.csv")

In [6]:
test_features=[]
test_labels = []
for i in range(10000):
    test_features.append(np.array(data_test.iloc[i][1:]).reshape(28,28,1))
    test_labels.append(data_test.iloc[i][0])

In [7]:
test_features = np.array(test_features).reshape(-1,28,28,1)
train_features = np.array(train_features).reshape(-1,28,28,1)
test_labels = np.array(test_labels).reshape(-1,1)
print("Test Features : ",test_features.shape)
print("Test Labels : ",test_labels.shape)

Test Features :  (10000, 28, 28, 1)
Test Labels :  (10000, 1)


In [8]:
train_features = train_features / 255.0
test_features = test_features / 255.0

In [9]:
# импортируем нашу библиотеку
import widelearning as wdl

In [10]:
# сгенерируем ядро вертикального типа размерности 5*5
k1 = wdl.vert([-2, -1, 0, 1, 2])

-2,-1,0,1,2,
-2,-1,0,1,2,
-2,-1,0,1,2,
-2,-1,0,1,2,
-2,-1,0,1,2,


In [13]:
# скопируем ядро в текстовый файл k1.txt и воспользовавшись функцией txt_kernel() подготовим его для установки
# в структуру нейронной сети

weights1 = wdl.txt_kernel('k1.txt')
weights1 = np.array(weights1)

In [14]:
# урезанная структура
# убрали обучение сверточного ядра первого слоя, установили собственное ядро, сгенерированное на предыдущем шаге
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(1,(5,5),activation='relu',input_shape=(28,28,1), trainable=False, use_bias=False))
model.add(keras.layers.Conv2D(32,(2,2),activation='relu'))
model.add(keras.layers.MaxPool2D((2,2)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(10,activation='softmax'))

model.compile(optimizer=keras.optimizers.Adam(0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.layers[0].set_weights([weights1])

ep = 7
history = model.fit(train_features,
                    train_labels,
                    batch_size=64,
                    validation_batch_size=1,
                    validation_data = (test_features,test_labels),
                    epochs = ep)

2022-08-23 16:21:09.761613: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-08-23 16:21:09.780006: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz


Epoch 1/7


2022-08-23 16:21:10.089986: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-08-23 16:21:10.197031: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


938/938 [==============================] - 10s 10ms/step - loss: 0.7635 - accuracy: 0.7340 - val_loss: 0.3753 - val_accuracy: 0.8632
Epoch 2/7
938/938 [==============================] - 9s 10ms/step - loss: 0.4093 - accuracy: 0.8536 - val_loss: 0.3380 - val_accuracy: 0.8804
Epoch 3/7
938/938 [==============================] - 9s 10ms/step - loss: 0.3663 - accuracy: 0.8668 - val_loss: 0.3205 - val_accuracy: 0.8847
Epoch 4/7
938/938 [==============================] - 10s 11ms/step - loss: 0.3320 - accuracy: 0.8780 - val_loss: 0.3057 - val_accuracy: 0.8878
Epoch 5/7
938/938 [==============================] - 9s 10ms/step - loss: 0.3142 - accuracy: 0.8839 - val_loss: 0.3008 - val_accuracy: 0.8924
Epoch 6/7
938/938 [==============================] - 9s 10ms/step - loss: 0.2886 - accuracy: 0.8931 - val_loss: 0.3012 - val_accuracy: 0.8924
Epoch 7/7
938/938 [==============================] - 9s 10ms/step - loss: 0.2686 - accuracy: 0.8997 - val_loss: 0.2839 - val_accuracy: 0.8954


In [15]:
# добавим к единственному ядру в первом сверточном слое ядро диагонального типа, сгенерировав его с
# помощью функции diag() и сохранив в файл k2.txt

k2 = wdl.diag(5)


0,1,1,2,2,
-1,0,1,1,2,
-1,-1,0,1,1,
-2,-1,-1,0,1,
-2,-2,-1,-1,0,

2,2,1,1,0,
2,1,1,0,-1,
1,1,0,-1,-1,
1,0,-1,-1,-2,
0,-1,-1,-2,-2,

In [16]:
weights2 = wdl.txt_kernel('k2.txt')

In [17]:
weights2

[[[[0]], [[1]], [[1]], [[2]], [[2]]],
 [[[-1]], [[0]], [[1]], [[1]], [[2]]],
 [[[-1]], [[-1]], [[0]], [[1]], [[1]]],
 [[[-2]], [[-1]], [[-1]], [[0]], [[1]]],
 [[[-2]], [[-2]], [[-1]], [[-1]], [[0]]]]

In [18]:
weights = wdl.add_kernel('k2.txt', [[[[0]], [[1]], [[1]], [[2]], [[2]]],
 [[[-1]], [[0]], [[1]], [[1]], [[2]]],
 [[[-1]], [[-1]], [[0]], [[1]], [[1]]],
 [[[-2]], [[-1]], [[-1]], [[0]], [[1]]],
 [[[-2]], [[-2]], [[-1]], [[-1]], [[0]]]])

weights

[[[[0, 0]], [[1, 1]], [[1, 1]], [[2, 2]], [[2, 2]]],
 [[[-1, -1]], [[0, 0]], [[1, 1]], [[1, 1]], [[2, 2]]],
 [[[-1, -1]], [[-1, -1]], [[0, 0]], [[1, 1]], [[1, 1]]],
 [[[-2, -2]], [[-1, -1]], [[-1, -1]], [[0, 0]], [[1, 1]]],
 [[[-2, -2]], [[-2, -2]], [[-1, -1]], [[-1, -1]], [[0, 0]]]]

In [19]:
weights = np.array(weights)

In [20]:

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(2,(5,5),activation='relu',input_shape=(28,28,1), trainable=False, use_bias=False))
model.add(keras.layers.Conv2D(32,(2,2),activation='relu'))
model.add(keras.layers.MaxPool2D((2,2)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(10,activation='softmax'))

model.compile(optimizer=keras.optimizers.Adam(0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.layers[0].set_weights([weights])

ep = 7
history = model.fit(train_features,
                    train_labels,
                    batch_size=64,
                    validation_batch_size=1,
                    validation_data = (test_features,test_labels),
                    epochs = ep)

Epoch 1/7
938/938 [==============================] - 9s 9ms/step - loss: 0.7635 - accuracy: 0.7294 - val_loss: 0.3861 - val_accuracy: 0.8612
Epoch 2/7
938/938 [==============================] - 9s 9ms/step - loss: 0.4193 - accuracy: 0.8492 - val_loss: 0.3429 - val_accuracy: 0.8774
Epoch 3/7
938/938 [==============================] - 9s 9ms/step - loss: 0.3736 - accuracy: 0.8675 - val_loss: 0.3296 - val_accuracy: 0.8777
Epoch 4/7
938/938 [==============================] - 9s 9ms/step - loss: 0.3303 - accuracy: 0.8801 - val_loss: 0.3217 - val_accuracy: 0.8859
Epoch 5/7
938/938 [==============================] - 9s 9ms/step - loss: 0.3043 - accuracy: 0.8879 - val_loss: 0.3075 - val_accuracy: 0.8861
Epoch 6/7
938/938 [==============================] - 9s 9ms/step - loss: 0.2732 - accuracy: 0.9004 - val_loss: 0.2996 - val_accuracy: 0.8915
Epoch 7/7
938/938 [==============================] - 9s 9ms/step - loss: 0.2616 - accuracy: 0.9043 - val_loss: 0.3134 - val_accuracy: 0.8888
